# Project: Decoding Molecules From Fingerprints.
## Group Members:
### Qi Chen, e-mail: gusqichr@student.gu.se
### Nils Dunlop, e-mail: gusdunlni@student.gu.se
### Francisco Alejandro Erazo Piza, e-mail: guserafr@student.gu.se
***

In [1]:
import os
import pandas as pd
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem import PandasTools
import numpy as np
from joblib import Parallel, delayed
from rdkit.DataStructs import TanimotoSimilarity
import subprocess
from typing import List, Tuple
import subprocess
from concurrent.futures import ThreadPoolExecutor, as_completed
from rdkit import Chem
import random
from typing import List, Tuple
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

### Define directories
***

In [2]:
# Set user directory
USER_DIR = os.path.expanduser('~')

# Set the project root directory
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Define the input and output directories
INPUT_DIR = os.path.join(PROJECT_ROOT, 'data/')
OUTPUT_DIR = os.path.join(PROJECT_ROOT, 'data/bit_flipping_nn_review_2')

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)


### COX2 and Janus Data
***

In [3]:
# Read COX2_SMILES.csv
cox2_df = pd.read_csv('../data/COX2_SMILES.csv', delimiter=';')
print(f"COX2 shape: {cox2_df.shape}")

# Read Janus_SMILES.csv
janus_df = pd.read_csv('../data/Janus_SMILES.csv', delimiter=';')
print(f"Janus shape: {janus_df.shape}")

COX2 shape: (21, 2)
Janus shape: (12, 2)


In [4]:
# Add molecule column to the dataframes
PandasTools.AddMoleculeColumnToFrame(cox2_df, smilesCol='SMILES', molCol='Molecule')
PandasTools.AddMoleculeColumnToFrame(janus_df, smilesCol='SMILES', molCol='Molecule')

In [5]:
cox2_df.head()

,title,SMILES,Molecule
0,Parecoxib,CCC(=O)NS(=O)(=O)c1ccc(cc1)c2c(C)onc2c3ccccc3,<rdkit.Chem.rdchem.Mol object at 0x7fd61076e730>
1,Anitrazafen,COc1ccc(cc1)c2nnc(C)nc2c3ccc(OC)cc3,<rdkit.Chem.rdchem.Mol object at 0x7fd61076e7a0>
2,Celecoxib,Cc1ccc(cc1)c2cc(nn2c3ccc(cc3)S(=O)(=O)N)C(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7fd61076e810>
3,Cimicoxib,COc1ccc(cc1F)c2c(Cl)ncn2c3ccc(cc3)S(=O)(=O)N,<rdkit.Chem.rdchem.Mol object at 0x7fd61076e8f0>
4,Deracoxib,COc1ccc(cc1F)c2cc(nn2c3ccc(cc3)S(=O)(=O)N)C(F)F,<rdkit.Chem.rdchem.Mol object at 0x7fd61076e960>


In [6]:
janus_df.head()

,title,SMILES,Molecule
0,Tofacitinib,C[C@@H]1CCN(C[C@@H]1N(C)c2ncnc3[nH]ccc23)C(=O)CC#N,<rdkit.Chem.rdchem.Mol object at 0x7fd61076f1b0>
1,Ruxolitinib,N#CC[C@H](C1CCCC1)n2cc(cn2)c3ncnc4[nH]ccc34,<rdkit.Chem.rdchem.Mol object at 0x7fd61076f220>
2,Oclacitinib,CNS(=O)(=O)C[C@@H]1CC[C@H](CC1)N(C)c2ncnc3[nH]ccc23,<rdkit.Chem.rdchem.Mol object at 0x7fd61076f290>
3,Baricitinib,CCS(=O)(=O)N1CC(CC#N)(C1)n2cc(cn2)c3ncnc4[nH]ccc34,<rdkit.Chem.rdchem.Mol object at 0x7fd61076f300>
4,Upadacitinib,CC[C@@H]1CN(C[C@@H]1c2cnc3cnc4[nH]ccc4n23)C(=O)NCC(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7fd61076f370>


In [7]:
def generate_fingerprint(mol):
    """
    Generate ECFP4 fingerprint and sparse representation for a molecule.
    """
    if mol is not None:
        morgan_generator = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)
        fp = morgan_generator.GetFingerprint(mol)
        bit_vector = np.array(list(fp.ToBitString())).astype(int)
        sparse_representation = list(np.where(bit_vector == 1)[0])
        sparse_representation_str = ' '.join(map(str, sparse_representation))
        return bit_vector, sparse_representation_str
    return None, None

In [8]:
# Apply the fingerprint generation function to COX2 DataFrame
cox2_results = Parallel(n_jobs=-1)(delayed(generate_fingerprint)(mol) for mol in cox2_df['Molecule'])
cox2_bit_vectors, cox2_sparse_fingerprints = zip(*cox2_results)

# Assign the new columns to the COX2 DataFrame
cox2_df['FingerprintBits'] = list(cox2_bit_vectors)
cox2_df['SparseFingerprintBits'] = list(cox2_sparse_fingerprints)

# Apply the fingerprint generation function to Janus DataFrame
janus_results = Parallel(n_jobs=-1)(delayed(generate_fingerprint)(mol) for mol in janus_df['Molecule'])
janus_bit_vectors, janus_sparse_fingerprints = zip(*janus_results)

# Assign the new columns to the Janus DataFrame
janus_df['FingerprintBits'] = list(janus_bit_vectors)
janus_df['SparseFingerprintBits'] = list(janus_sparse_fingerprints)


In [9]:
cox2_df.head()

,title,SMILES,Molecule,FingerprintBits,SparseFingerprintBits
0,Parecoxib,CCC(=O)NS(=O)(=O)c1ccc(cc1)c2c(C)onc2c3ccccc3,<rdkit.Chem.rdchem.Mol object at 0x7fd61076e730>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",41 80 179 248 294 307 319 323 331 350 361 378 389 461 471 476 502 624 650 656 715 736 807 835 883 896 898 1045 1057 1088 1141 1152 1160 1199 1366 1380 1476 1542 1722 1746 1747 1750 1873 1917
1,Anitrazafen,COc1ccc(cc1)c2nnc(C)nc2c3ccc(OC)cc3,<rdkit.Chem.rdchem.Mol object at 0x7fd61076e7a0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",123 139 269 322 323 357 378 454 471 695 718 781 831 841 896 978 1057 1126 1160 1164 1380 1536 1722 1750 1855 1873
2,Celecoxib,Cc1ccc(cc1)c2cc(nn2c3ccc(cc3)S(=O)(=O)N)C(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7fd61076e810>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",114 233 235 319 350 368 378 650 715 753 809 833 875 896 935 1057 1160 1171 1197 1328 1380 1399 1434 1440 1446 1453 1476 1489 1527 1607 1692 1722 1750 1823 1825 1852 1873 1920 1928 1956
3,Cimicoxib,COc1ccc(cc1F)c2c(Cl)ncn2c3ccc(cc3)S(=O)(=O)N,<rdkit.Chem.rdchem.Mol object at 0x7fd61076e8f0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...]",27 94 118 233 249 289 319 339 350 378 561 564 650 695 699 704 715 724 746 833 841 875 879 896 932 935 1057 1160 1171 1197 1260 1380 1445 1452 1476 1489 1536 1607 1649 1683 1750 1825 1840 1873 1928 1945 1970
4,Deracoxib,COc1ccc(cc1F)c2cc(nn2c3ccc(cc3)S(=O)(=O)N)C(F)F,<rdkit.Chem.rdchem.Mol object at 0x7fd61076e960>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...]",1 94 118 233 249 259 305 319 350 378 501 650 695 699 715 833 841 875 896 935 991 1050 1057 1160 1171 1197 1328 1380 1399 1405 1440 1476 1489 1536 1607 1649 1692 1750 1780 1825 1840 1846 1852 1873 1928 1945 1956 1970


In [10]:
janus_df.head()

,title,SMILES,Molecule,FingerprintBits,SparseFingerprintBits
0,Tofacitinib,C[C@@H]1CCN(C[C@@H]1N(C)c2ncnc3[nH]ccc23)C(=O)CC#N,<rdkit.Chem.rdchem.Mol object at 0x7fd61076f1b0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",51 72 80 87 259 361 369 378 417 479 489 506 560 590 650 652 674 678 711 739 788 790 806 807 881 926 935 1009 1019 1026 1057 1089 1114 1152 1163 1171 1309 1325 1357 1380 1384 1452 1480 1506 1750 1764 1807 1810 1853 1859 1860 1873 1917 1973 2009
1,Ruxolitinib,N#CC[C@H](C1CCCC1)n2cc(cn2)c3ncnc4[nH]ccc34,<rdkit.Chem.rdchem.Mol object at 0x7fd61076f220>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",1 43 80 87 276 323 335 339 360 378 464 479 489 674 711 741 790 806 808 890 926 935 944 1019 1026 1028 1051 1089 1114 1152 1160 1161 1171 1256 1325 1348 1357 1380 1384 1452 1506 1535 1634 1706 1739 1750 1810 1873 1876 1897 1978
2,Oclacitinib,CNS(=O)(=O)C[C@@H]1CC[C@H](CC1)N(C)c2ncnc3[nH]ccc23,<rdkit.Chem.rdchem.Mol object at 0x7fd61076f290>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",29 51 72 80 87 169 259 312 350 369 378 479 489 565 650 711 788 806 809 881 904 926 1019 1025 1026 1057 1114 1152 1154 1163 1171 1325 1357 1380 1430 1452 1454 1476 1506 1524 1561 1671 1750 1810 1873 2018
3,Baricitinib,CCS(=O)(=O)N1CC(CC#N)(C1)n2cc(cn2)c3ncnc4[nH]ccc34,<rdkit.Chem.rdchem.Mol object at 0x7fd61076f300>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",43 80 87 141 225 226 276 294 323 350 378 464 479 489 585 602 650 674 675 711 723 741 790 806 872 923 926 935 1026 1057 1060 1089 1114 1152 1160 1171 1256 1357 1380 1384 1452 1476 1502 1506 1535 1700 1750 1810 1817 1873 1876 1897 1915 1971
4,Upadacitinib,CC[C@@H]1CN(C[C@@H]1c2cnc3cnc4[nH]ccc4n23)C(=O)NCC(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7fd61076f370>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",45 48 80 114 187 197 213 218 226 294 311 378 455 479 503 510 546 548 607 638 650 711 739 806 807 817 819 887 926 935 1009 1019 1039 1057 1092 1104 1114 1123 1152 1225 1228 1231 1380 1452 1453 1454 1506 1535 1549 1750 1799 1826 1873 1917 1928 2009


### Extract Nearby Neighbors of parecoxib
***

In [11]:
parecoxib = cox2_df[cox2_df['title'] == 'Parecoxib']
anitrazafen = cox2_df[cox2_df['title'] == 'Anitrazafen']
celecoxib = cox2_df[cox2_df['title'] == 'Celecoxib']
cimicoxib = cox2_df[cox2_df['title'] == 'Cimicoxib']
deracoxib = cox2_df[cox2_df['title'] == 'Deracoxib']

### Flip Bits
***

In [12]:
def flip_bits(vector: List[int], positions: List[int]) -> List[int]:
    """Flip the bits at the given positions in the vector."""
    flipped_vector = vector.copy()
    for position in positions:
        flipped_vector[position] = 1 - flipped_vector[position]
    return flipped_vector

def batched_random_bit_flipping(fingerprint: List[int], batch_size: int) -> List[Tuple[List[int], List[int]]]:
    """Perform batched random bit flipping on the fingerprint."""
    total_bits = len(fingerprint)
    all_positions = list(range(total_bits))
    random.shuffle(all_positions)
    
    results = []
    for i in range(0, total_bits, batch_size):
        batch_positions = all_positions[i:i+batch_size]
        flipped_fingerprint = flip_bits(fingerprint, batch_positions)
        results.append((batch_positions, flipped_fingerprint))
    
    return results

def sparse_to_dense(sparse_fingerprint: List[int], size: int = 2048) -> List[int]:
    """Convert sparse fingerprint to dense fingerprint."""
    dense = [0] * size
    for bit in sparse_fingerprint:
        dense[bit] = 1
    return dense

def dense_to_sparse(dense_fingerprint: List[int]) -> List[int]:
    """Convert dense fingerprint to sparse fingerprint."""
    return [i for i, bit in enumerate(dense_fingerprint) if bit == 1]

def generate_flipped_fingerprints(row: pd.Series, batch_size: int) -> List[dict]:
    """Generate batched random bit flipped fingerprints for a single row."""
    dense_fingerprint = row['FingerprintBits']
    sparse_fingerprint = row['SparseFingerprintBits']

    flipped_results = batched_random_bit_flipping(dense_fingerprint, batch_size)

    new_rows = []
    for positions, flipped_dense in flipped_results:
        flipped_sparse = dense_to_sparse(flipped_dense)
        new_row = {
            'title': row['title'],
            'SMILES': row['SMILES'],
            'FingerprintBits': dense_fingerprint,
            'SparseFingerprintBits': sparse_fingerprint,
            'FingerprintBitsFlipped': flipped_dense,
            'SparseFingerprintBitsFlipped': flipped_sparse,
            'FlippedBitPositions': positions
        }
        new_rows.append(new_row)

    return new_rows


def process_dataframe(df: pd.DataFrame, batch_size: int) -> pd.DataFrame:
    """Process the entire dataframe and generate batched random bit flipped fingerprints."""
    all_new_rows = []
    for _, row in df.iterrows():
        new_rows = generate_flipped_fingerprints(row, batch_size)
        all_new_rows.extend(new_rows)

    return pd.DataFrame(all_new_rows)

In [13]:
# 2 random bit flips
parecoxib_df_2_random_flips = process_dataframe(parecoxib, 2)
anitrazafen_df_2_random_flips = process_dataframe(anitrazafen, 2)
celecoxib_df_2_random_flips = process_dataframe(celecoxib, 2)
cimicoxib_df_2_random_flips = process_dataframe(cimicoxib, 2)
deracoxib_df_2_random_flips = process_dataframe(deracoxib, 2)

In [14]:
# 4 random bit flips
parecoxib_df_4_random_flips = process_dataframe(parecoxib, 4)
anitrazafen_df_4_random_flips = process_dataframe(anitrazafen, 4)
celecoxib_df_4_random_flips = process_dataframe(celecoxib, 4)
cimicoxib_df_4_random_flips = process_dataframe(cimicoxib, 4)
deracoxib_df_4_random_flips = process_dataframe(deracoxib, 4)

In [15]:
# 8 random bit flips
parecoxib_df_8_random_flips = process_dataframe(parecoxib, 8)
anitrazafen_df_8_random_flips = process_dataframe(anitrazafen, 8)
celecoxib_df_8_random_flips = process_dataframe(celecoxib, 8)
cimicoxib_df_8_random_flips = process_dataframe(cimicoxib, 8)
deracoxib_df_8_random_flips = process_dataframe(deracoxib, 8)

In [16]:
# 128 random bit flips
parecoxib_df_128_random_flips = process_dataframe(parecoxib, 128)
anitrazafen_df_128_random_flips = process_dataframe(anitrazafen, 128)
celecoxib_df_128_random_flips = process_dataframe(celecoxib, 128)
cimicoxib_df_128_random_flips = process_dataframe(cimicoxib, 128)
deracoxib_df_128_random_flips = process_dataframe(deracoxib, 128)

In [17]:
# 1024 random bit flips
parecoxib_df_1024_random_flips = process_dataframe(parecoxib, 1024)
anitrazafen_df_1024_random_flips = process_dataframe(anitrazafen, 1024)
celecoxib_df_1024_random_flips = process_dataframe(celecoxib, 1024)
cimicoxib_df_1024_random_flips = process_dataframe(cimicoxib, 1024)
deracoxib_df_1024_random_flips = process_dataframe(deracoxib, 1024)

### MolForge predictions on flipped fingerprints
***


In [18]:
# Set user directory
USER_DIR = os.path.expanduser('~')

# Path to the conda environment to MolForge
CONDA_ENV_PATH = os.path.join(USER_DIR, 'conda/envs/molforge/bin/python')
MOLFORGE_DIR = os.path.join(USER_DIR, 'MolForge')
MOLFORGE_SCRIPT_PATH = os.path.join(USER_DIR, 'MolForge/predict.py')
WORKERS = 4

In [19]:
def convert_spaced_smiles(spaced_smiles):
    compact_smiles = spaced_smiles.replace(" ", "")
    mol = Chem.MolFromSmiles(compact_smiles)
    if mol is not None:
        return Chem.MolToSmiles(mol, isomericSmiles=False)
    else:
        return "Invalid SMILES string"

In [20]:
def run_molforge(fp_input):
    # Ensure fp_input is a string
    if isinstance(fp_input, list):
        fp_input_str = ' '.join(map(str, fp_input))
    else:
        fp_input_str = fp_input
    
    command = [
        CONDA_ENV_PATH,
        MOLFORGE_SCRIPT_PATH,
        "--fp=ECFP4",
        "--model_type=smiles",
        f"--input={fp_input_str}"
    ]
    
    try:
        result = subprocess.run(
            command,
            capture_output=True,
            text=True,
            check=True,
            cwd=MOLFORGE_DIR,
            env=os.environ.copy()
        )
    except subprocess.CalledProcessError as e:
        print(f"Error running MolForge: {e}")
        print(f"Error output: {e.stderr}")
        return "Error", "Error"
    
    spaced_smiles = None
    device_used = None
    output_lines = result.stdout.splitlines()
    
    for line in output_lines:
        if "Result:" in line:
            spaced_smiles = line.split("Result:")[1].strip()
        if "rank :" in line:
            device_used = line.split("rank :")[1].strip()
    
    if spaced_smiles:
        compact_smiles = convert_spaced_smiles(spaced_smiles)
    else:
        compact_smiles = "No Result"

    return compact_smiles, device_used

In [21]:
def process_row(row):
    fp_input = row['SparseFingerprintBitsFlipped']
    compact_smiles, device_used = run_molforge(fp_input)
    return compact_smiles, device_used

def process_dataframe(df, max_rows=None):
    if max_rows:
        df = df.head(max_rows)
    
    total = len(df)
    with ThreadPoolExecutor(max_workers=WORKERS) as executor:
        futures = {executor.submit(process_row, row): idx for idx, row in df.iterrows()}
        
        for i, future in enumerate(as_completed(futures)):
            idx = futures[future]
            result, device_used = future.result()
            df.at[idx, 'Generated_SMILES'] = result
            df.at[idx, 'Device_Used'] = device_used
            if (i + 1) % 100 == 0 or (i + 1) == total:
                print(f"Processed molecule {i + 1}/{total}")
    
    return df

In [22]:
parecoxib_df_2_random_flips = process_dataframe(parecoxib_df_2_random_flips)
parecoxib_df_2_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'parecoxib_df_2_random_flips.parquet'), index=False)

parecoxib_df_4_random_flips = process_dataframe(parecoxib_df_4_random_flips)
parecoxib_df_4_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'parecoxib_df_4_random_flips.parquet'), index=False)

parecoxib_df_8_random_flips = process_dataframe(parecoxib_df_8_random_flips)
parecoxib_df_8_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'parecoxib_df_8_random_flips.parquet'), index=False)

parecoxib_df_128_random_flips = process_dataframe(parecoxib_df_128_random_flips)
parecoxib_df_128_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'parecoxib_df_128_random_flips.parquet'), index=False)

parecoxib_df_1024_random_flips = process_dataframe(parecoxib_df_1024_random_flips)
parecoxib_df_1024_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'parecoxib_df_1024_random_flips.parquet'), index=False)

Processed molecule 100/1024
Processed molecule 200/1024
Processed molecule 300/1024
Processed molecule 400/1024
Processed molecule 500/1024
Processed molecule 600/1024
Processed molecule 700/1024
Processed molecule 800/1024
Processed molecule 900/1024
Processed molecule 1000/1024
Processed molecule 1024/1024
Processed molecule 100/512
Processed molecule 200/512
Processed molecule 300/512
Processed molecule 400/512
Processed molecule 500/512
Processed molecule 512/512
Processed molecule 100/256


[17:12:55] SMILES Parse Error: unclosed ring for input: 'CCC(=O)NS(=O)(=O)C1=CC=C(C=C1)C2=NOC3=C2CC4=C(C(=NO4)C5=CC=CC=C5)C6=CC=CC=C6'


Processed molecule 200/256
Processed molecule 256/256


[17:13:59] SMILES Parse Error: unclosed ring for input: 'CC/C(=C\1/C(=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=O)C#C=C=O)/CC'
[17:13:59] SMILES Parse Error: unclosed ring for input: 'CC/C(=C/1\C(=C(/C(=C/2\C(=C(C(=C(CC)CC)C(=C3C=CON3)C2=C(CC)CC)C4=CC=C(O4)CC)CC)/CC)\O)C(=C(O)O)CC)/[NH2+][CH]CC'
[17:14:05] SMILES Parse Error: unclosed ring for input: 'CCC(=O)NS(=O)(=O)C1=CC=C(C=C1)C2=C3C=CC=C4C3=CP(=N4)(C2)SNP5(=C6C=CC=C7C6=C8C=CC=CC8=N5)N[Si](C)(C)C'
[17:14:08] SMILES Parse Error: unclosed ring for input: 'CC/C(=C\1/C=CC(=C([N+]#N)O)C=C1)/NS(=O)(=O)C2=CC=C(C=C2)[Si](CC)(CC)CC3=NN=C4C=CC(=C5C=CC(=NN5)C6=CC=CC=C6)C=C4[Si](CC)(CC)CC'


Processed molecule 16/16
Processed molecule 2/2


[17:14:10] SMILES Parse Error: syntax error while parsing: C1=C(C(=C2C1=CC(=C3C2=C(S3)C(=O)OCC4[CH]C(=C5C4=CC(=C6SC(=C7SC(=CC(=C8S7)C(=C6S5)C(=C8)C)C(=O)OCS9)SP(S5)S)SP(S7)SC)S7)S5)S7)C(S7)C(
[17:14:10] SMILES Parse Error: Failed parsing SMILES 'C1=C(C(=C2C1=CC(=C3C2=C(S3)C(=O)OCC4[CH]C(=C5C4=CC(=C6SC(=C7SC(=CC(=C8S7)C(=C6S5)C(=C8)C)C(=O)OCS9)SP(S5)S)SP(S7)SC)S7)S5)S7)C(S7)C(' for input: 'C1=C(C(=C2C1=CC(=C3C2=C(S3)C(=O)OCC4[CH]C(=C5C4=CC(=C6SC(=C7SC(=CC(=C8S7)C(=C6S5)C(=C8)C)C(=O)OCS9)SP(S5)S)SP(S7)SC)S7)S5)S7)C(S7)C('
[17:14:10] SMILES Parse Error: extra open parentheses for input: 'C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(CO)C(CO)CO)(CO)CO)[Se])CO)(F)[Se])O)O)O)O)O)O)O)(F)[Se]CO)O)O)O)O)O)'


In [23]:
anitrazafen_df_2_random_flips = process_dataframe(anitrazafen_df_2_random_flips)
anitrazafen_df_2_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'anitrazafen_df_2_random_flips.parquet'), index=False)

anitrazafen_df_4_random_flips = process_dataframe(anitrazafen_df_4_random_flips)
anitrazafen_df_4_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'anitrazafen_df_4_random_flips.parquet'), index=False)

anitrazafen_df_8_random_flips = process_dataframe(anitrazafen_df_8_random_flips)
anitrazafen_df_8_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'anitrazafen_df_8_random_flips.parquet'), index=False)

anitrazafen_df_128_random_flips = process_dataframe(anitrazafen_df_128_random_flips)
anitrazafen_df_128_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'anitrazafen_df_128_random_flips.parquet'), index=False)

anitrazafen_df_1024_random_flips = process_dataframe(anitrazafen_df_1024_random_flips)
anitrazafen_df_1024_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'anitrazafen_df_1024_random_flips.parquet'), index=False)

Processed molecule 100/1024
Processed molecule 200/1024
Processed molecule 300/1024
Processed molecule 400/1024
Processed molecule 500/1024
Processed molecule 600/1024
Processed molecule 700/1024
Processed molecule 800/1024
Processed molecule 900/1024
Processed molecule 1000/1024
Processed molecule 1024/1024


[17:24:36] SMILES Parse Error: unclosed ring for input: 'CC1=NC2=NC(=C(N=N1)C3=CC=C(C=C3)OC)C4=CC=C(C=C4)OC'
[17:24:39] SMILES Parse Error: unclosed ring for input: 'CC1=NC(=C(N=N1)C2=CC=C(C=C2)OC)S3(C4=NC(=C(N=N4)C5=CC=C(C=C5)OC)C6=CC=C(C=C6)OC)OC'
[17:24:39] SMILES Parse Error: unclosed ring for input: 'CC1=NC2=NC(=C(N=N2)C3=CC=C(C=C3)OC)C4=CC=C(C=C4)OC'


Processed molecule 100/512
Processed molecule 200/512
Processed molecule 300/512
Processed molecule 400/512


[17:28:08] SMILES Parse Error: unclosed ring for input: 'CC1=NC2=NC(=C(N=N2)C3=CC=C(C=C3)OC)C4=CC=C(C=C4)OC'


Processed molecule 500/512
Processed molecule 512/512


...

[17:30:24] SMILES Parse Error: unclosed ring for input: 'CC1=NC2=NC(=C(N=N1)C3=CC=C(C=C3)OC)C4=C(N=C(N=N4)C)C5=CC=C(C=C5)OC'
[17:30:25] SMILES Parse Error: unclosed ring for input: 'CC1=NC2=NC(=C(N=N1)C3=CC=C(C=C3)OC)C4=C(N=C(N=N4)OC)C5=CC=C(C=C5)OC'
[17:30:25] SMILES Parse Error: unclosed ring for input: 'CC1=NC2=NC(=C(N=N1)C3=CC=C(C=C3)OC)N=NC4=NC(=C(N=N4)C5=CC=C(C=C5)OC)C6=CC=C(C=C6)OC'
[17:30:27] SMILES Parse Error: unclosed ring for input: 'CC1=NC2=NC(=C(N=N1)C3=CC=C(C=C3)OC)C4=CC=C(C=C4)OC'
[17:30:27] SMILES Parse Error: unclosed ring for input: 'CC1=NC2=NC(=C(N=N1)C3=CC=C(C=C3)OC)N=NC4=NC(=C(N=N4)C5=CC=C(C=C5)OC)C6=CC=C(C=C6)OC'
[17:30:32] SMILES Parse Error: unclosed ring for input: 'CC1=NC2=NC(=C(N=N1)C3=CC=C(C=C3)OC)C4=NC(=C(N=N4)C5=CC=C(C=C5)OC)C6=CC=C(C=C6)OC'
[17:30:35] SMILES Parse Error: unclosed ring for input: 'CC1=NC2=NC(=C(N=N1)C3=CC=C(C=C3)OC)N=NC4=NC(=C(N=N4)C5=CC=C(C=C5)OC)C6=CC=C(C=C6)OC'
[17:30:44] SMILES Parse Error: unclosed ring for input: 'CC1=NC2=C(N=N1)C3=

Processed molecule 256/256


[17:30:59] SMILES Parse Error: unclosed ring for input: 'B(C1=CC=C(C=C1)OC)(C2=CC=C(C=C2)OC)C3=C4C5=C(C(=NN=N5)[Se]C6=CC=C(C=C6)OC)N=N4'
[17:30:59] SMILES Parse Error: unclosed ring for input: 'COC1=CC=C(C=C1)C2=C(N=NC(=N2)[N+](C3=CC=C(C=C3)OC)(C4=CC=C(C=C4)OC)C(=C5C=C6C(=NN=N6)C(=C7C=NN=N7)O)C(=[Se])OC)C8=CC=CCC=C8'
[17:30:59] SMILES Parse Error: unclosed ring for input: 'COC1=CC=C(C=C1)C2=C(C(=NN=N2)C3=CC=C(C=C3)OC)C4=CC=C(C=C4)C5=C6C(=C7C=CC(=C8N=CC=N8)C=C7)N=N6'
[17:31:02] SMILES Parse Error: unclosed ring for input: '[B](C1=C(C(=C=C=C2C(C(C(=C=C3C(C(C(=C=C4C(C(C(=C=C5C(=NN=N5)C6=CC=C(C=C6)OC)OC)S)S)C(=C=C4)OC)S)OC)S)S)C(=C2)OC)S)S)S'
[17:31:04] SMILES Parse Error: unclosed ring for input: 'C[Si](C)(C)/C/1=C/2\C(=C(N=N2)C3=CC=C(C=C3)OC)[Se]/C=C/4\C=CC(=C/C=C/5\C(=NOO5)C6=CC=C(C=C6)OC)[Se]C=C1'
[17:31:05] SMILES Parse Error: unclosed ring for input: 'COC1=CC=C(C=C1)C2=C(N=NC(=N2)C3=CC=C(C=C3)OC)C4=CC(C(C(C4)O)OC(=O)C5=C6C(=NN=N6)C7=COC=C7)OO'
[17:31:07] SMILES Parse Error: unclosed 

Processed molecule 16/16
Processed molecule 2/2


[17:31:09] SMILES Parse Error: extra open parentheses for input: 'C1=C2C=C(C=C1C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(C(F)(F)F)C(C(F)(F)F)C(C(F)(F)F)C(F)(F)F)C(F)(F)F)C(F)(F)F)(F)F)(F)F)(F)F)F)F)F)F)(F)F)'
[17:31:09] SMILES Parse Error: unclosed ring for input: 'C(=C\1/C(=C/2\C(=C(C(=C2SP(S1)SC(P(S)P(S)P(P(P(P(P(P(P(P(F)F)S)P(F)F)P(F)F)S)S)SP(P(P(P(F)F)P(F)F)S)S5)S4)S3)S5=S=S)P(S5)S)S5)S5)'


In [24]:
celecoxib_df_2_random_flips = process_dataframe(celecoxib_df_2_random_flips)
celecoxib_df_2_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'celecoxib_df_2_random_flips.parquet'), index=False)

celecoxib_df_4_random_flips = process_dataframe(celecoxib_df_4_random_flips)
celecoxib_df_4_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'celecoxib_df_4_random_flips.parquet'), index=False)

celecoxib_df_8_random_flips = process_dataframe(celecoxib_df_8_random_flips)
celecoxib_df_8_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'celecoxib_df_8_random_flips.parquet'), index=False)

celecoxib_df_128_random_flips = process_dataframe(celecoxib_df_128_random_flips)
celecoxib_df_128_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'celecoxib_df_128_random_flips.parquet'), index=False)

celecoxib_df_1024_random_flips = process_dataframe(celecoxib_df_1024_random_flips)
celecoxib_df_1024_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'celecoxib_df_1024_random_flips.parquet'), index=False)


Processed molecule 100/1024
Processed molecule 200/1024
Processed molecule 300/1024
Processed molecule 400/1024
Processed molecule 500/1024
Processed molecule 600/1024
Processed molecule 700/1024
Processed molecule 800/1024
Processed molecule 900/1024
Processed molecule 1000/1024
Processed molecule 1024/1024
Processed molecule 100/512
Processed molecule 200/512
Processed molecule 300/512
Processed molecule 400/512
Processed molecule 500/512
Processed molecule 512/512
Processed molecule 100/256
Processed molecule 200/256
Processed molecule 256/256


[17:48:35] Explicit valence for atom # 34 C, 5, is greater than permitted
[17:48:35] SMILES Parse Error: unclosed ring for input: 'CS(C)(C)CCC(C1=CC=CC=C1)(C2=CC3=CC4=CC=C5C(N4C6=NN=C7C=C(C=CC7=C6)C8=C9C=CC=CC9=C[N+]8=NN=N8)C[NH3+]=CC=C5C[NH3+])OON'
[17:48:36] SMILES Parse Error: unclosed ring for input: 'C1=NC=C(C=C1)C2=C3C(=C4C=CC(=C5N4N=CN=CN5)C=C6N=CN=CN6)N=PN=C7N=CC(=CN=CN2)C=C7'
[17:48:41] SMILES Parse Error: unclosed ring for input: 'C[C]1[CH][C]2[CH][C]([CH][C]2[CH]1)C(=C3[CH][CH][CH][C]3C(=O)[CH]4[CH][C]5[CH][CH][C]([CH]5)[C]6[C]7[C]([CH][CH][C]7[CH][CH][CH]6)C=C8[C]7N([CH][CH][CH]8)N=C9N=CN=NN9)C(=O)[C]1[CH][CH][C]([CH][CH]1)C(F)(F)F'
[17:48:41] SMILES Parse Error: unclosed ring for input: 'C[Si](C)(C)N1C=C2C(=C1)N=NC3=CC(=CN23)S(=O)(=O)C4=CC5=C(C=C4)N=NC6=CC(=CN([CH]65)C(N)(N)N)N=N5'
[17:48:41] SMILES Parse Error: extra open parentheses for input: '[B][N+](=C1C=C(C=C1C2=C(C(=C(C(=C2)C(=C3C=C(C=C3)C4=C(C(=C(C(=C4)C(=C5C(=CC(=N5)C(F)(F)F)C#N)C([B])(F)[B])C([B])([B])[B])C([B])(

Processed molecule 16/16


[17:48:46] SMILES Parse Error: extra close parentheses while parsing: C1=C2C(=CC1=C3C(=CC(=C4C(=CC(=C5C=C(C(=C6C(=CC(=N6)C7=CC(=CC(=C2)[N-]7)[S-])[S-])C(C5([S-])[S-])[S-])[S-])[CH]4)[S-])[S-])[CH][CH][CH][CH][CH]5)[S-])C(C([S-])(C[S-])[S-])[S-]
[17:48:46] SMILES Parse Error: Failed parsing SMILES 'C1=C2C(=CC1=C3C(=CC(=C4C(=CC(=C5C=C(C(=C6C(=CC(=N6)C7=CC(=CC(=C2)[N-]7)[S-])[S-])C(C5([S-])[S-])[S-])[S-])[CH]4)[S-])[S-])[CH][CH][CH][CH][CH]5)[S-])C(C([S-])(C[S-])[S-])[S-]' for input: 'C1=C2C(=CC1=C3C(=CC(=C4C(=CC(=C5C=C(C(=C6C(=CC(=N6)C7=CC(=CC(=C2)[N-]7)[S-])[S-])C(C5([S-])[S-])[S-])[S-])[CH]4)[S-])[S-])[CH][CH][CH][CH][CH]5)[S-])C(C([S-])(C[S-])[S-])[S-]'


Processed molecule 2/2


[17:48:46] SMILES Parse Error: extra close parentheses while parsing: C1(=C2C(=C3C1=C4C(=C5C(=C(S4)P(C)C)C(C(C5(SP(C)C)P(C)C)P(C)C)(C(C(C(F)(F)F)OC(=S)C(C(F)(F)F)C(F)(F)F)C(F)(F)F)SP(C)C)C)C)C=S3)P(C
[17:48:46] SMILES Parse Error: Failed parsing SMILES 'C1(=C2C(=C3C1=C4C(=C5C(=C(S4)P(C)C)C(C(C5(SP(C)C)P(C)C)P(C)C)(C(C(C(F)(F)F)OC(=S)C(C(F)(F)F)C(F)(F)F)C(F)(F)F)SP(C)C)C)C)C=S3)P(C' for input: 'C1(=C2C(=C3C1=C4C(=C5C(=C(S4)P(C)C)C(C(C5(SP(C)C)P(C)C)P(C)C)(C(C(C(F)(F)F)OC(=S)C(C(F)(F)F)C(F)(F)F)C(F)(F)F)SP(C)C)C)C)C=S3)P(C'


In [25]:
cimicoxib_df_2_random_flips = process_dataframe(cimicoxib_df_2_random_flips)
cimicoxib_df_2_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'cimicoxib_df_2_random_flips.parquet'), index=False)

cimicoxib_df_4_random_flips = process_dataframe(cimicoxib_df_4_random_flips)
cimicoxib_df_4_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'cimicoxib_df_4_random_flips.parquet'), index=False)

cimicoxib_df_8_random_flips = process_dataframe(cimicoxib_df_8_random_flips)
cimicoxib_df_8_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'cimicoxib_df_8_random_flips.parquet'), index=False)

cimicoxib_df_128_random_flips = process_dataframe(cimicoxib_df_128_random_flips)
cimicoxib_df_128_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'cimicoxib_df_128_random_flips.parquet'), index=False)

cimicoxib_df_1024_random_flips = process_dataframe(cimicoxib_df_1024_random_flips)
cimicoxib_df_1024_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'cimicoxib_df_1024_random_flips.parquet'), index=False)

Processed molecule 100/1024
Processed molecule 200/1024
Processed molecule 300/1024
Processed molecule 400/1024
Processed molecule 500/1024
Processed molecule 600/1024
Processed molecule 700/1024
Processed molecule 800/1024
Processed molecule 900/1024
Processed molecule 1000/1024
Processed molecule 1024/1024
Processed molecule 100/512
Processed molecule 200/512
Processed molecule 300/512
Processed molecule 400/512
Processed molecule 500/512
Processed molecule 512/512
Processed molecule 100/256
Processed molecule 200/256
Processed molecule 256/256


[18:05:59] SMILES Parse Error: unclosed ring for input: 'CO[Si](C1=CC(=CC(=C1)C2=C(N=CN2C3=CC=C(C=C3)C4=C(N=CN4C5=CC(=C(SN)S(=O)(=O)[O-])[Si](OC)(OC)OC)Cl)Cl)S(ON)(ON)ON)(OC)OC'
[18:06:02] SMILES Parse Error: unclosed ring for input: 'CON1C=NN(C1=C2C=C3C(=C2)C(=C4N3C(=C5N4C(=C6C=C7C=C8C=CC(=CC8=C7ON9C=NN=C9)S(=O)(=O)N)C(=NN)Cl)Cl)OOC)CN'
[18:06:05] Conflicting single bond directions around double bond at index 30.
[18:06:05]   BondStereo set to STEREONONE and single bond directions set to NONE.
[18:06:07] SMILES Parse Error: unclosed ring for input: '[B][NH2+]N1C=C(N=C1)C[NH+]2C=C(C=C2)C3=C(N=CN3OS(=O)(=O)OC4=CC5=CC(=C4)[S+](C=C5)C)C6=CC7=CC(=C=O)[S+](C7=C([B])[B])N'
[18:06:07] SMILES Parse Error: unclosed ring for input: 'C/C(=C/1\C=C2C=CC=C3C2=C1N=C3)/C4=CC(=CC=C4)S(=C5C=C6C=CC(=C7C6=CN=N7)S(=O)(=O)OC)(=O)OCC8=CC(=CN=C8)S(=O)(=O)N'
[18:06:07] SMILES Parse Error: unclosed ring for input: 'CC1(CC=CCS1(=O)C2=CC=C(C=C2)N3C=NC(=C3C4=CC(=C5C(=NC=[N+]5S(=O)(=O)OC)Cl)C6=CC(=NS(=O)(=O)OC)C=CS

Processed molecule 16/16
Processed molecule 2/2


[18:06:09] SMILES Parse Error: extra close parentheses while parsing: C1=C2C=C3C(=C4C1=C5C=C(C=C(C5=C4C(C(C(=C6C=C7C=C(C=C7C=C6)C(C(=C8C=C(C=N8)C(=C9C=N9)[S-])[S-])[S-])[S-])[S-])(C(F)(F)[S-])C(=C[S-])[S-])[S-])[S-])[S-])[S-])[CH]2
[18:06:09] SMILES Parse Error: Failed parsing SMILES 'C1=C2C=C3C(=C4C1=C5C=C(C=C(C5=C4C(C(C(=C6C=C7C=C(C=C7C=C6)C(C(=C8C=C(C=N8)C(=C9C=N9)[S-])[S-])[S-])[S-])[S-])(C(F)(F)[S-])C(=C[S-])[S-])[S-])[S-])[S-])[S-])[CH]2' for input: 'C1=C2C=C3C(=C4C1=C5C=C(C=C(C5=C4C(C(C(=C6C=C7C=C(C=C7C=C6)C(C(=C8C=C(C=N8)C(=C9C=N9)[S-])[S-])[S-])[S-])[S-])(C(F)(F)[S-])C(=C[S-])[S-])[S-])[S-])[S-])[S-])[CH]2'


In [26]:
deracoxib_df_2_random_flips = process_dataframe(deracoxib_df_2_random_flips)
deracoxib_df_2_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'deracoxib_df_2_random_flips.parquet'), index=False)

deracoxib_df_4_random_flips = process_dataframe(deracoxib_df_4_random_flips)
deracoxib_df_4_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'deracoxib_df_4_random_flips.parquet'), index=False)

deracoxib_df_8_random_flips = process_dataframe(deracoxib_df_8_random_flips)
deracoxib_df_8_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'deracoxib_df_8_random_flips.parquet'), index=False)

deracoxib_df_128_random_flips = process_dataframe(deracoxib_df_128_random_flips)
deracoxib_df_128_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'deracoxib_df_128_random_flips.parquet'), index=False)

deracoxib_df_1024_random_flips = process_dataframe(deracoxib_df_1024_random_flips)
deracoxib_df_1024_random_flips.to_parquet(os.path.join(OUTPUT_DIR, 'deracoxib_df_1024_random_flips.parquet'), index=False)

Processed molecule 100/1024
Processed molecule 200/1024
Processed molecule 300/1024
Processed molecule 400/1024
Processed molecule 500/1024
Processed molecule 600/1024
Processed molecule 700/1024


[18:13:23] SMILES Parse Error: extra close parentheses while parsing: COC1=C(C=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(C4=NN(C(=C4)C5=CC(=C(C=C5)OC)F)C6=CC=C(C=C6)S(=O)(=O)N)C7=CC(C=C(C=C7)F)F)F)F
[18:13:23] SMILES Parse Error: Failed parsing SMILES 'COC1=C(C=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(C4=NN(C(=C4)C5=CC(=C(C=C5)OC)F)C6=CC=C(C=C6)S(=O)(=O)N)C7=CC(C=C(C=C7)F)F)F)F' for input: 'COC1=C(C=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(C4=NN(C(=C4)C5=CC(=C(C=C5)OC)F)C6=CC=C(C=C6)S(=O)(=O)N)C7=CC(C=C(C=C7)F)F)F)F'


Processed molecule 800/1024
Processed molecule 900/1024
Processed molecule 1000/1024
Processed molecule 1024/1024
Processed molecule 100/512
Processed molecule 200/512


[18:18:40] SMILES Parse Error: unclosed ring for input: 'COC1=C(C=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(C4=NN(C(=C4)C5=CC(=C(C=C5)OC)F)C6=CC=C(C=C6)S(=O)(=O)N)C7=NN)F'


Processed molecule 300/512
Processed molecule 400/512
Processed molecule 500/512
Processed molecule 512/512
Processed molecule 100/256
Processed molecule 200/256
Processed molecule 256/256


[18:23:34] Explicit valence for atom # 6 N, 4, is greater than permitted
[18:23:35] SMILES Parse Error: extra close parentheses while parsing: C1=CC(=CC=C1C2=NC(N=N2)S(=O)(=O)C3=CC(=C(C=C3)OC(C4=COC(=N4)S(=O)(=O)OC)F)C(F)F)S(=O)(=O)ON5C(=NC(C5=O)(C(F)F)C(F)F)F)F
[18:23:35] SMILES Parse Error: Failed parsing SMILES 'C1=CC(=CC=C1C2=NC(N=N2)S(=O)(=O)C3=CC(=C(C=C3)OC(C4=COC(=N4)S(=O)(=O)OC)F)C(F)F)S(=O)(=O)ON5C(=NC(C5=O)(C(F)F)C(F)F)F)F' for input: 'C1=CC(=CC=C1C2=NC(N=N2)S(=O)(=O)C3=CC(=C(C=C3)OC(C4=COC(=N4)S(=O)(=O)OC)F)C(F)F)S(=O)(=O)ON5C(=NC(C5=O)(C(F)F)C(F)F)F)F'
[18:23:36] SMILES Parse Error: unclosed ring for input: 'CON1C2=CC(=CC(=N2)N3C4(N3S1(N)OC)N5C(=N4)C=C6C57N(N=NN7S(N)(OC)OC)S(=O)(=O)OC)C(C)C'


Processed molecule 16/16
Processed molecule 2/2


[18:23:44] SMILES Parse Error: extra close parentheses while parsing: C(C1=C(C(=C(C1=C2C(=C(C(=C2P(C3=CC=C(C=C3)P(C4=CC=C(C=C4)C#C)C5=CC=C(C=C5)[S-])[S-])P([S-])[S-])[S-])[S-])P([S-])[S-])P([S-])[S-])[S-])P([S-])[S-])[S-]
[18:23:44] SMILES Parse Error: Failed parsing SMILES 'C(C1=C(C(=C(C1=C2C(=C(C(=C2P(C3=CC=C(C=C3)P(C4=CC=C(C=C4)C#C)C5=CC=C(C=C5)[S-])[S-])P([S-])[S-])[S-])[S-])P([S-])[S-])P([S-])[S-])[S-])P([S-])[S-])[S-]' for input: 'C(C1=C(C(=C(C1=C2C(=C(C(=C2P(C3=CC=C(C=C3)P(C4=CC=C(C=C4)C#C)C5=CC=C(C=C5)[S-])[S-])P([S-])[S-])[S-])[S-])P([S-])[S-])P([S-])[S-])[S-])P([S-])[S-])[S-]'
[18:23:44] SMILES Parse Error: unclosed ring for input: 'C1=C2C=C3C(=C4C1=C(C(=C4)[Se]C(C(C(C(C(C(C(C(C(C(C(C(=S)N)[Se]C(=S)N)[Se]C(=S)N)[Se]C(=S)N)[Se]C(=S)N)OC(=S)N)[Se]5)[Se]C)OC(=S)N)[Se]C)OC(=S)[Se]3)[Se]C)[Se]C)N2'
